In [ ]:
import pandas as pd 
import numpy as np
from utils_ import utilss
from utils_ import ImpliedVar
from models.Core import Core
import matplotlib.pyplot as plt 
import warnings 
from utils_ import IVTermStructure
import tqdm
from models import FiniteDifference
warnings.filterwarnings('ignore')


# Upload, clean and process the Data


In [ ]:
dfx = utilss.process_and_merge_data(['data/aapl_2016_2020.csv'], ['TB3MS'], start_date='2016-01-01', end_date='2023-04-30')

In [ ]:
dfx['C_VOLUME']=  dfx['C_VOLUME'].astype('float')
dfx['P_VOLUME']=  dfx['P_VOLUME'].astype('float')


In [ ]:
df.columns

In [ ]:
dfx=dfx.dropna()
df_sub=dfx[(dfx['C_VOLUME']>=50) ].iloc[:1000, :]

In [ ]:
df_sub

# Recalculate the greeks 

In [ ]:
opml_instance = Core(dfx)
df=opml_instance.add_greeks_to_df()
df=opml_instance.add_bs_call_price()
df['QUOTE_DATE'] = pd.to_datetime(df['QUOTE_DATE'], format='%Y-%m-%d')
df['Option_Contract_ID'] = ("OP" + df['STRIKE'].astype(str) + '-'+ df['EXPIRE_DATE'].dt.month.astype(str).str.zfill(2) +'-'+  df['EXPIRE_DATE'].dt.year.astype(str))


In [ ]:
iv_term_structure = IVTermStructure.IVTermStructure(df)
term_structures_df = iv_term_structure.get_iv_term_structure()
print(term_structures_df)

In [ ]:
plt.figure(figsize=(15,10))
df.groupby(['QUOTE_DATE'])[['C_THETA_BS', 'C_THETA']].median().plot()
df.groupby(['QUOTE_DATE'])[['C_IV', 'C_AS_PERCT_OF_UNDER']].median().plot()
df.groupby(['QUOTE_DATE'])[['C_RHO_BS', 'C_RHO']].std().plot()
df[['C_RHO_BS', 'C_RHO']].quantile(0.6)

In [ ]:
df.groupby(['QUOTE_DATE'])[['C_VOLUME']].median().plot()
df.groupby(['QUOTE_DATE'])[['P_VOLUME']].median().plot()


In [ ]:
df.groupby(['QUOTE_DATE'])['STRIKE_DISTANCE_PCT'].min().plot()
df.groupby(['QUOTE_DATE'])['C_AS_PERCT_OF_UNDER'].min().plot(label='c as %')
plt.legend()

In [ ]:
df[['C_RHO_BS', 'C_RHO']].quantile(0.998)

In [ ]:
df.groupby(['QUOTE_DATE'])[['C_MIDPRICE', 'BS_CALL_PRICE']].mean().plot()


In [ ]:
df[df['QUOTE_DATE']=='2017-09-15'].groupby(['STRIKE'])['EXPIRE_DATE'].mean()


In [ ]:
df.Option_Contract_ID.unique()

In [ ]:
df[df['Option_Contract_ID']=='OP97.5-03-2016'].groupby('QUOTE_DATE')['UNDERLYING_LAST'].mean().plot()


In [ ]:
df[df['Option_Contract_ID']=='OP97.5-03-2016'].BS_CALL_PRICE.plot()
df[df['Option_Contract_ID']=='OP97.5-03-2016'].C_ASK.plot()
df[df['Option_Contract_ID']=='OP97.5-03-2016'].C_BID.plot()


In [ ]:
df[df['Option_Contract_ID']=='OP120.0-01-2017']

## Visualize the volatility surface  

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a copy of the dataframe
copy_df = df.copy()

# Create empty lists to store data
dates = []
C_IV_values = []
strike_values = []
DTE_values = []

# Iterate over unique dates
for date in copy_df['QUOTE_DATE'].unique():
    daily_mat = copy_df[copy_df['QUOTE_DATE'] == date] 
    
    C_IV = daily_mat['C_IV'].values
    K = daily_mat['STRIKE'].values
    DTE = daily_mat['DTE'].values
    
    

# Create a 3D plot
    fig = plt.figure(figsize=(10,10))
    
    ax = fig.add_subplot(111, projection='3d')
# Plot the vol surface
    ax.scatter(DTE, K, C_IV)

# Set labels and title
    ax.set_xlabel('DAYS TO MATURITY')
    ax.set_ylabel('Moneyness')
    ax.set_zlabel('Implcit Call Volatility')
    ax.set_title(date)

# Show the plot
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
import pandas as pd
import numpy as np

# Assuming 'df' is pre-defined with columns 'QUOTE_DATE', 'C_IV', 'STRIKE', 'DTE'

# Make a copy of the DataFrame to ensure original data is not modified
copy_df = df.copy()

# Setup figure and 3D axis for the plot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Set labels for axes
ax.set_xlabel('Days to Expiry')
ax.set_ylabel('Strike Price')
ax.set_zlabel('Implied Call Volatility')

# Prepare for animation by sorting unique dates
unique_dates = sorted(copy_df['QUOTE_DATE'].unique())

# Initialization function for the animation
def init():
    ax.set_xlabel('Days to Expiry')
    ax.set_ylabel('Strike Price')
    ax.set_zlabel('Implied Call Volatility')
    return fig,

# Function to update the plot for each frame
def update(frame):
    ax.clear()  # Clear current data
    # Set labels again because clear() removes them
    ax.set_xlabel('Days to Expiry')
    ax.set_ylabel('Strike Price')
    ax.set_zlabel('Implied Call Volatility')
    date = unique_dates[frame]
    daily_data = copy_df[copy_df['QUOTE_DATE'] == date]
    DTE = daily_data['DTE'].values
    Strike = daily_data['STRIKE'].values
    IV = daily_data['C_IV'].values
    ax.scatter(DTE, Strike, IV, c=IV, cmap='viridis')  # Color mapped by IV
    ax.set_title(f'Volatility Surface for {date}')
    return fig,

# Create animation
ani = FuncAnimation(fig, update, frames=len(unique_dates), init_func=init, blit=False, interval=200)

# Save the animation as a GIF
ani.save('volatility_surface_animation.gif', writer=PillowWriter(fps=2))

# If you want to display the animation in a Jupyter notebook, you can uncomment:
# from IPython.display import HTML
# HTML(ani.to_html5_video())


# Deduce the risk free rate [slow to run]

In [ ]:
opml_instance = OPML(df)
df=opml_instance.deduce_r_fallback_multithreaded(initial_guess=0.01)

# Create a tensor

In [ ]:
opml_instance = Core(df)
tensor = opml_instance.df_to_tensor_x(index_col='QUOTE_DATE', sort_cols=['QUOTE_DATE', 'STRIKE'])
print(tensor.shape)  # Prints the shape of the tensor


In [ ]:
plt.figure(figsize=(10,8))
df['C_VOLUME']=df['C_VOLUME'].astype('float')
df['P_VOLUME']=df['P_VOLUME'].astype('float')

df[df['QUOTE_DATE']=="2020-01-31"]['P_VOLUME'].hist(bins=40)

In [ ]:
tensor[0,0:2,:]

In [ ]:
df.sort_values(by=['QUOTE_DATE', 'STRIKE']).iloc[0,:]

In [ ]:
df.isna().sum()

# Finite Difference

In [ ]:
x = df[df['DTE'] != 0].copy().iloc[:5000,:]

pricer = FiniteDifference.FiniteDifference(x)
pricer.price_options(N=50, M=50, american=True, option_type='call')  # Set american=False for European options


In [ ]:
df.STRIKE_DISTANCE.head()

In [ ]:
x.groupby(['QUOTE_DATE'])['BS_CALL_PRICE', 'C_ASK', 'Option_Price_FD'].median().plot()